# <a href="https://thetahat.ru/courses/ph-ds-2024-aut">Phystech@DataScience</a>
## Семинар 8 (Часть 2)

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

In [ ]:
# Bot check

# HW_ID: phds_sem8_2
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [ ]:
#!pip install catboost

In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.datasets import fetch_california_housing

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

sns.set(font_scale=1.8, palette='Set2')

##  Сравнение различных бустингов и случайного леса

###  №1 Задача регрессии на предсказание цены квартир

Исследуем зависимость качества предсказаний градиентного бустинга и случайного леса в зависимости от числа базовых моделей на примере задаче регрессии. Для случайного леса будем использовать класс `RandomForestRegressor` библиотеки `sklearn`. В качестве таргета будет предсказание цены квартир в Калифорнии в зависимости от различных параметров.

In [ ]:
housing = fetch_california_housing()
print("Признаки:",housing.feature_names)
X, y = housing.data, housing.target

Разобьём данные на обучающую выборку и на тестовую, выделив на тест 25% данных.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
def plot_compare_estimators(estimator_labels, param_grid, train_metrics,
                            test_metrics, param_label='', metrics_label='',
                            title=''):
    '''Функция для построения графиков зависимости целевой метрики
    от некоторого параметра модели на обучающей и на валидационной
    выборке.

    :param estimator_labels: массив названий моделей
    :param param_grid: значения исследуемого параметра
    :param train_metrics: массив, где i-ый элемент -
        это массив значений метрики на обучающей выборке для i-ой модели
    :param test_metrics: массив, где i-ый элемент -
        это массив значений метрики на валидационной выборке для i-ой модели
    :param param_label: названия параметра
    :param metrics_label: название метрики
    :param title: заголовок для графика
    '''

    plt.figure(figsize=(20, 7))

    for estimator_id in range(len(estimator_labels)):
        label = estimator_labels[estimator_id]
        plt.plot(
            param_grid, train_metrics[estimator_id],
            label=f'{label} train', linewidth=3
        )
        plt.plot(
            param_grid, test_metrics[estimator_id],
            label=f'{label} test', linewidth=3
        )

    plt.legend()
    plt.xlabel(param_label)
    plt.ylabel(metrics_label)
    plt.title(title, fontsize=20)

#### Обучим случайный лес для разного числа деревьев и посчитаем `mse`.

In [ ]:
rf_mse_train = []  # метрика на трейне
rf_mse_test = []  # метрика на тесте
n_estimators_grid = range(1, 300, 10)

for n_estimators in tqdm(n_estimators_grid):
    rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1)
    rf.fit(X_train, y_train)
    rf_mse_train.append(mse(y_train, rf.predict(X_train)))
    rf_mse_test.append(mse(y_test, rf.predict(X_test)))

Обучим градиентный бустинг для разного числа деревьев и посчитаем `mse`.

In [ ]:
boosting_mse_train = []
boosting_mse_test = []

for n_estimators in tqdm(n_estimators_grid):
    gb = GradientBoostingRegressor(n_estimators=n_estimators)
    gb.fit(X_train, y_train)
    boosting_mse_train.append(mse(y_train, gb.predict(X_train)))
    boosting_mse_test.append(mse(y_test, gb.predict(X_test)))

Обучим СatBoost для разного числа деревьев и посчитаем `mse`.

In [ ]:
catboost_mse_train = []
catboost_mse_test = []

for n_estimators in tqdm(n_estimators_grid):
    catboost = CatBoostRegressor(iterations=n_estimators, verbose=0)
    catboost.fit(X_train, y_train)
    catboost_mse_train.append(mse(y_train, catboost.predict(X_train)))
    catboost_mse_test.append(mse(y_test, catboost.predict(X_test)))

Обучим LGBM для разного числа деревьев и посчитаем `mse`.

In [ ]:
lgbm_mse_train = []
lgbm_mse_test = []

for n_estimators in tqdm(n_estimators_grid):
    lgbm = LGBMRegressor(n_estimators=n_estimators)
    lgbm.fit(X_train, y_train)
    lgbm_mse_train.append(mse(y_train, lgbm.predict(X_train)))
    lgbm_mse_test.append(mse(y_test, lgbm.predict(X_test)))

Построим график зависимости `mse` от количества базовых моделей.

In [ ]:
plot_compare_estimators(
    ['random forest', 'boosting', "CatBoost", "LGBM"], n_estimators_grid,
    [rf_mse_train, boosting_mse_train, catboost_mse_train, lgbm_mse_train],
    [rf_mse_test, boosting_mse_test, catboost_mse_test,lgbm_mse_test],
    'Количество деревьев', 'MSE',
    'Сравнение случайного леса и бустингов'
)

**Вывод:**

С увеличением числа деревьев лосс у случайного леса снижается быстрее, но быстро достигает плато. Также можно заметить, что случайный лес склонен к переобучению — его лосс на тестовых данных стабильно выше, чем на обучающих.

В случае бустинга лосс уменьшается медленнее, но значения на трейне и тесте остаются примерно одинаковыми, что указывает на отсутствие явного переобучения. К концу нашей сетки лосс для бустинга всё ещё продолжает снижаться. Однако ближе к концу и у бустинга начинают расходиться значения лосса на тренировочных и тестовых данных, что указывает на начало переобучения.

#### Сравним времена обучения и предсказания случайного леса и бустингов с параметрами по умолчанию:

Градиентный бустинг

In [ ]:
%%timeit

regressor_boosting = GradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
regressor_boosting = GradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
%%timeit

regressor_boosting.predict(X_test)

Случайный лес

In [ ]:
%%timeit

regressor_forest = RandomForestRegressor(n_jobs=-1).fit(X_train, y_train)

In [ ]:
regressor_forest = RandomForestRegressor(n_jobs=-1).fit(X_train, y_train)

In [ ]:
%%timeit

regressor_forest.predict(X_test)

СatBoost regressor

In [ ]:
%%timeit

catboost = CatBoostRegressor(silent=True).fit(X_train, y_train)

In [ ]:
catboost = CatBoostRegressor(silent=True).fit(X_train, y_train)

In [ ]:
%%timeit

catboost.predict(X_test)

LGBM regressor

In [ ]:
%%timeit

lgbm = LGBMRegressor(silent=True).fit(X_train, y_train)

In [ ]:
lgbm = LGBMRegressor(silent=True).fit(X_train, y_train)

In [ ]:
%%timeit

lgbm.predict(X_test)

**Вывод:**

Обучение бустинга занимает примерно вдвое больше времени по сравнению со случайным лесом.

Однако предсказание у бустинга проходит быстрее — примерно в 3.5 раза.

### №2 Задача на классификацию студентов

Сделаем аналогичный эксперимент с датасетом для классификации на 3 класса студентов (отчислен, закончил обучение, продолжает обучение). Целевой колонкой является `Target`.

In [ ]:
students = pd.read_csv("students_dropout_succes.csv")
students.head()

In [ ]:
X = students.drop(columns = ["Target"])
y = students["Target"]

Разобьём данные на обучающую и тестовую выборки, под тестовую выделим 30%. Обязательно зафиксируйте `random_state` для воспроизводимости результатов.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Обучим случайный лес для разного числа деревьев и посчитаем метрику `f1_score`.

In [ ]:
rf_f1_train = []
rf_f1_test = []
n_estimators_grid = range(1, 300, 25)

for n_estimators in tqdm(n_estimators_grid):
    rf = RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1)
    rf.fit(X_train, y_train)
    rf_f1_train.append(f1_score(y_train, rf.predict(X_train), average='weighted'))
    rf_f1_test.append(f1_score(y_test, rf.predict(X_test), average='weighted'))

Обучим градиентный бустинг для разного числа деревьев и посчитаем метрику `f1_score`.

In [ ]:
catboost_f1_train = []
catboost_f1_test = []

for n_estimators in tqdm(n_estimators_grid):
    catboost = CatBoostClassifier(n_estimators=n_estimators, silent=True)
    catboost.fit(X_train, y_train)
    catboost_f1_train.append(f1_score(y_train, catboost.predict(X_train), average='weighted'))
    catboost_f1_test.append(f1_score(y_test, catboost.predict(X_test), average='weighted'))

Обучим LGBM для разного числа деревьев и посчитаем метрику `f1_score`.

In [ ]:
lgbm_f1_train = []
lgbm_f1_test = []

for n_estimators in tqdm(n_estimators_grid):
    lgbm = LGBMClassifier(n_estimators=n_estimators)
    lgbm.fit(X_train, y_train)
    lgbm_f1_train.append(f1_score(y_train, lgbm.predict(X_train), average='weighted'))
    lgbm_f1_test.append(f1_score(y_test, lgbm.predict(X_test), average='weighted'))

Построим график зависимости `f1_score` от количества базовых моделей.

In [ ]:
plot_compare_estimators(
    ['random forest', 'boosting', "catboost", "lgbm"],
    n_estimators_grid,
    [rf_f1_train, boosting_f1_train, catboost_f1_train, lgbm_f1_train],
    [rf_f1_test, boosting_f1_test, catboost_f1_test, lgbm_f1_test],
    'Количество деревьев',
    'f1_score',
    'Сравнение случайного леса и бустингов'
)

**Вывод:**

Ситуация, похоже, напоминает пункт о регрессии:

Для случайного леса точность на обучающей выборке быстро достигла значения 1, в то время как на тестовой выборке она также быстро пришла к плато, не превышающему 1, что свидетельствует о переобучении.

В случае с бустингом точность растет медленнее. Изначально результаты на тестовой и обучающей выборках почти не различаются, но после 50 деревьев наблюдается значительное расхождение. Ближе к 300 деревьям точность на обучающей выборке также достигает значения 1, а на тестовой — приближается к плато, аналогичному лесу. Тем не менее, есть вероятность, что она еще может немного вырасти.

## AutoML &mdash; LightAutoMl Framework

LAMA AutoML (Automated Machine Learning) — это система автоматизированного машинного обучения, разработанная для упрощения процесса создания и оптимизации моделей машинного обучения. Она автоматизирует следующее:

1. **Подготовка данных**. Автоматическая предобработка данных, включая нормализацию, заполнение пропущенных значений и другие преобразования.
2. **Выбор признаков**. Автоматический отбор наиболее значимых признаков для модели.
3. **Выбор модели**. Ансамблированный выбор комбинаций наилучших моделей.
4. **Настройка гиперпараметров**. Автоматическая оптимизация гиперпараметров.
5. **Оценка модели**. Автоматическая оценка производительности модели с использованием кросс-валидации и других методов.

то есть 70% процентов наших текущих задач

![image.png](attachment:image.png)

Подробнее можно прочитать тут: https://lightautoml.readthedocs.io/en/latest/pages/tutorials/Tutorial_1_basics.html

Для начала скачаем библиотеку (может занять много времени, можно попробовать ставить в Google Collab)

In [ ]:
!pip install -U lightautoml

Импортируем нужные нам библиотеки

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

Для LAMA целевая колонка задается по другому, поэтому разобьем наш датасет на train и test

In [ ]:
students = pd.read_csv("students_dropout_succes.csv")
train_data, test_data = train_test_split(students, test_size = 0.3, random_state=42)

Теперь зададим целевую колонку с помощью атрибута `roles`

In [ ]:
roles = {
    'target': 'Target', #целевая колонка тут Target
}

#### Создание объекта Task

Сначала нам нужно создать объект `Task` — класс, чтобы указать, какую задачу должна решать модель LightAutoML с определенными потерями и метрикой, если это необходимо (более подробную информацию можно найти в документации).

Доступные типы задач

1. **'binary'** - для бинарной классификации.
2. **'reg'** - для регрессии.
3. **'multiclass'** - для многоклассовой классификации.
4. **'multi:reg'** - для множественной регрессии.
5. **'multilabel'** - для многомарковой классификации.

In [ ]:
automl = TabularAutoML(
    task = Task('multiclass'), #введите, какую задачу мы собираемся решать
    timeout = 600, #введите количество времени, в течении которого будет обучаться LAMA
)

Теперь запустим обучение:

In [ ]:
%%time
out_of_fold_predictions = automl.fit_predict(train_data, roles = roles, verbose = 1)

Какие модели используются в итоговом предсказании и какими весами?

**Ответ:**

В итоговом предсказании LightAutoML используются несколько базовых моделей, таких как CatBoost, LightGBM, и XGBoost. Веса моделей определяются в процессе обучения на валидационных данных и применяются в методе стекирования.  

Сделаем обратный маппинг классов (из LabelEncoding в слова)

In [ ]:
mapping = automl.reader.class_mapping
def map_class(x):
    return mapping[x]
mapped = np.vectorize(map_class)

Посмотрим на результаты обучения

In [ ]:
%%time

test_predictions = automl.predict(test_data)

print(f'Train F1 score: {f1_score(mapped(train_data[roles["target"]].values), np.argmax(out_of_fold_predictions.data, axis = 1), average = "weighted")}')
print(f'Test F1 score: {f1_score(mapped(test_data[roles["target"]].values), np.argmax(test_predictions.data, axis = 1), average = "weighted")}')

Посмотрите на важность фичей:

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

**Выводы:**

1. LightAutoML использует комбинацию CatBoost, LightGBM и XGBoost для улучшения точности предсказаний.

2. Итоговые предсказания формируются на основе весов, которые определяются в процессе обучения, что повышает устойчивость модели.
